# This notebook is going to download eHydro bathymetry data from the USACE ArcGIS REST repository, as well as retrieve cloud masked imagery of the same location, at the same time, for training of Satellite Derived Bathymetry model(s) for the National Channel Framework (NCF)

In [19]:
import requests
import os
import zipfile
import numpy as np
import ee
from osgeo import gdal
import rasterio
from pykrige.ok import OrdinaryKriging
from rasterio.mask import mask
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import fiona
from scipy.interpolate import griddata
from pyproj import Proj, transform
from pyproj import Transformer
from shapely.geometry import Point, Polygon, MultiPoint

In [2]:
ee.Initialize(project = '') ##enter your project name here as a string to initialize exchanges with ee api

# Functions

In [3]:
def get_gee_search_dates(time):
    date_obj = datetime.utcfromtimestamp(time / 1000)
    return ((date_obj - timedelta(days=1)).strftime('%Y-%m-%d'), (date_obj + timedelta(days=1)).strftime('%Y-%m-%d'))

def ehydro_date_convert(time):
    return datetime.utcfromtimestamp(time / 1000).strftime('%Y-%m-%d')

def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(destination, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(destination)}",
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
            bar.update(len(chunk))

def get_s2_sr_cld_col(aoi, start_date, end_date, cloud_filter):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    combined_coll = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

    return combined_coll

def interpolate_bathymetry(surveyname, resolution, storage_dir):
    folder_path = os.path.join(DOWNLOAD_DIR, surveyname)
    input_shapefile = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.gdb')][0]
    output_raster = "/tmp/bathy_interp.tif"  # Output raster file path
    clipped_raster = '/tmp/bath_clip.tif'
    resampled_raster = os.path.join(storage_dir, f'{surveyname}.tif')
    z_field = "depthMean"  # Attribute containing bathymetry or depth values
    resolution = 10  # Desired pixel resolution in meters

    gdf = gpd.read_file(input_shapefile, layer="Bathymetry_Vector")
    xmin, ymin, xmax, ymax = gdf.total_bounds  # Get the extent of the layer

    # Calculate raster width and height
    width = round((xmax - xmin) / resolution)
    height = round((ymax - ymin) / resolution)

    # --- Step 3: Create the Raster Using gdal.Grid ---
    gdal.Grid(
        output_raster,                # Output raster path
        input_shapefile,              # Input vector data
        format="GTiff",               # Output file format
        algorithm="invdist",          # Interpolation method (IDW)
        zfield=z_field,               # Attribute containing bathymetry values
        outputBounds=[xmin, ymin, xmax, ymax],  # Set bounds
        width=width,                  # Number of columns
        height=height,                # Number of rows
        layers="Bathymetry_Vector",   # Specify the layer
        z_multiply=-1                 # Flip depths to negative
    )
    
    # --- Step 4: Clip the Raster to the GDF Geometry ---
    # Combine all geometries into a single boundary
    geometry = [gdf.geometry.union_all()]

    # Open the created raster and clip it using the GDF boundary
    with rasterio.open(output_raster) as src:
        clipped_image, clipped_transform = mask(
            src, geometry, crop=True, nodata=np.nan
        )
        clipped_meta = src.meta.copy()
        clipped_meta.update({
            "driver": "GTiff",
            "height": clipped_image.shape[1],
            "width": clipped_image.shape[2],
            "transform": clipped_transform,
            "nodata": np.nan
        })

    # Save the clipped raster to a new file
    with rasterio.open(clipped_raster, "w", **clipped_meta) as dst:
        dst.write(clipped_image[0], 1)  # Access first band

    # --- Step 5: Resample the Clipped Raster to 10m Resolution ---
    gdal.Warp(
        resampled_raster,       # Output resampled raster path
        clipped_raster,         # Input clipped raster
        xRes=resolution,              # Set pixel size in x direction
        yRes=resolution,              # Set pixel size in y direction
        resampleAlg=gdal.GRA_Bilinear, # Bilinear interpolation for resampling
        targetAlignedPixels=True,     # Align pixels to the grid
        dstNodata=np.nan              # Set NoData value to NaN
    )

    os.remove(output_raster)
    os.remove(clipped_raster)
    print(f"Resampled raster saved to: {resampled_raster}")


# Densify lines and extract points
def extract_points_from_contours(gdf, spacing, z_field):
    points = []
    depths = []
    for idx, row in gdf.iterrows():
        line = row.geometry
        depth = row[z_field]
        # Sample points along the line at a regular interval
        for i in np.arange(0, line.length, spacing):
            point = line.interpolate(i)  # Interpolate point at distance `i`
            points.append(point)
            depths.append(depth)
    return points, depths





# Query bathy data
AVAILABLE FIELD NAMES:
- Field Name: OBJECTID, Type: esriFieldTypeOID
- Field Name: surveyjobidpk, Type: esriFieldTypeString
- Field Name: sdsid, Type: esriFieldTypeString
- Field Name: sdsfeaturename, Type: esriFieldTypeString
- Field Name: sdsmetadataid, Type: esriFieldTypeString
- Field Name: surveytype, Type: esriFieldTypeString
- Field Name: channelareaidfk, Type: esriFieldTypeString
- Field Name: dateuploaded, Type: esriFieldTypeDate
- Field Name: usacedistrictcode, Type: esriFieldTypeString
- Field Name: surveydatestart, Type: esriFieldTypeDate
- Field Name: surveydateend, Type: esriFieldTypeDate
- Field Name: sourcedatalocation, Type: esriFieldTypeString
- Field Name: sourceprojection, Type: esriFieldTypeString
- Field Name: mediaidfk, Type: esriFieldTypeString
- Field Name: projectedarea, Type: esriFieldTypeDouble
- Field Name: sdsfeaturedescription, Type: esriFieldTypeString
- Field Name: dateloadedenterprise, Type: esriFieldTypeDate
- Field Name: datenotified, Type: esriFieldTypeDate
- Field Name: sourcedatacontent, Type: esriFieldTypeString
- Field Name: plotsheetlocation, Type: esriFieldTypeString
- Field Name: sourceagency, Type: esriFieldTypeString
- Field Name: globalid, Type: esriFieldTypeGlobalID
- Field Name: Shape__Area, Type: esriFieldTypeDouble
- Field Name: Shape__Length, Type: esriFieldTypeDouble

For training the model, will probably want to include options for with:
- usace district
- time of year (date and season)
- NCF ID
- survey type (single vs dual beam; XC, BD, AD, etc.)

In [4]:
# initiate search parameters for eHydro

s2_cloud_cov = 20 ## percentage of clouds in sentinel-2 multispectral imagery, less means you see more surface
search_date = '2018-01-01'  # Date threshold, getting data from 2018 to present
usace_code = "CESWG"        # Galveston District (for now)

NUM_OF_QUERIES = 3          # number of iterations for the request to run
QUERY_TIME_DELAY = 2        # query time delay in seconds, used when requesting all features
URL = "https://services7.arcgis.com/n1YM8pTrFmm7L4hs/ArcGIS/rest/services/eHydro_Survey_Data/FeatureServer/0/query"
DOWNLOAD_DIR = f'/home/clay/Documents/SDB/{usace_code}/bathy'


In [ ]:
# Parameters for the initial query
params = {
    'where': f"surveydatestart >= '{search_date}' AND usacedistrictcode='{usace_code}'",
    'outFields': '*',  # Retrieve all fields
    'resultRecordCount': 2000,  # Maximum records per request
    'resultOffset': 0,  # Starting offset
    'f': 'json',  # Output format
    'outSR': '4326',  # Spatial reference
}

all_features = []

for i in range(NUM_OF_QUERIES):
    response = requests.get(URL, params=params)
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        if not features:
            break
        all_features.extend(features)
        params['resultOffset'] += params['resultRecordCount']
        print(f"Retrieved {len(features)} features.")
        time.sleep(QUERY_TIME_DELAY)  # Delay of 1 second
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

1. Extract date and aoi from the surveys for GEE
- plan is to iterate through the queries (probably by district code) and check to see if GEE has a corresponding Sentinel-2 image

In [9]:
geeinfo = {}
dates = []
for feature in all_features:
    dates.append(ehydro_date_convert(feature['attributes']['surveydatestart']))
    area = ee.Geometry.Polygon(feature['geometry']['rings'][0])

    date_tuple = get_gee_search_dates(feature['attributes']['surveydatestart'])

    geeinfo[feature['attributes']['surveyjobidpk']] = [area, date_tuple]
surveykeys = list(geeinfo.keys())

2. Iterate through responses and check if GEE has corresponding image(s)
- if not, the response will be deleted

In [10]:
for survey, items in geeinfo.items():
    aoi = items[0]
    dates = items[1]

    coll = get_s2_sr_cld_col(aoi, dates[0], dates[1], s2_cloud_cov)

    if coll.size().getInfo() > 0:
        geeinfo[survey].append(coll)

Get all surveys that have initial images

In [11]:
goodsurveys = []
for survey, items in geeinfo.items():
    if len(items) > 2:
        goodsurveys.append(survey)

if len(goodsurveys) == 0:
    print('No appropriate images were found')

3. Extract the eHydro bathy data download urls

In [14]:
bathyinfo = {}
for i, feature in enumerate(all_features):
    bathyinfo[surveykeys[i]] = feature['attributes']['sourcedatalocation']

4. Download the eHydro data locally for training models

In [ ]:
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

for survey in goodsurveys:
    file_path = os.path.join(DOWNLOAD_DIR, f"{survey}.zip")
    try:
        download_file(bathyinfo[survey], file_path)
    except Exception as e:
        print(f"Failed to download {bathyinfo[survey]}: {e}")

print('='*250)
print("All files downloaded.")

# Unzip downloaded data

In [5]:
zipnames = [f[:-4] for f in os.listdir(DOWNLOAD_DIR) if f.endswith('.zip')]
if len(zipnames) > 0:
    for name in zipnames:
        zipfile_path = os.path.join(DOWNLOAD_DIR, f'{name}.zip')
        with zipfile.ZipFile(zipfile_path,'r') as zip_ref:
            zip_ref.extractall(zipfile_path[:-4])
            os.remove(zipfile_path)
    surveynames = [f for f in os.listdir(DOWNLOAD_DIR) if not f.startswith('.')]
else:
    surveynames = [f for f in os.listdir(DOWNLOAD_DIR) if not f.startswith('.')]

# Get the .gdb files from the eHydro data
- will use the Bathymetry_Vector multipolygon shapefile to create bathymetry rasters at 10-meter spatial resolution to match with Sentinel-2. Each polygon in the shapefiles represents a mean depth

In [29]:
gdbinfo = {}
for name in surveynames:
    folder_path = os.path.join(DOWNLOAD_DIR, name)
    gdb_file = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.gdb')][0]
    layers = fiona.listlayers(gdb_file)
    bathyvector = gpd.read_file(gdb_file, layer='Bathymetry_Vector')
    # contours = gpd.read_file(gdb_file, layer="ElevationContour_ALL")

    gdbinfo[name] = bathyvector

In [30]:

def test_natural_neighbor_with_mask(gdf, output_file, resolution=10):
    """
    Test Natural Neighbor Interpolation with masking to polygon boundaries.
    Save the raster output reprojected to EPSG:4326.

    Args:
        gdf (GeoDataFrame): Input GeoDataFrame with 'geometry' and 'depthMean' columns.
        output_file (str): Path to save the generated raster (GeoTIFF).
        resolution (int): Resolution of the output raster (e.g., 10 for 10m).

    Returns:
        None: Saves raster to the specified file.
    """
    if gdf.crs is None or not gdf.crs.is_projected:
        raise ValueError("GeoDataFrame must have a defined and projected CRS.")

    # Step 1: Prepare the data for interpolation
    gdf["centroid"] = gdf.geometry.centroid
    x = gdf.centroid.x.values
    y = gdf.centroid.y.values
    z = gdf["depthMean"].values

    # Step 2: Determine the bounds and grid for rasterization
    xmin, ymin, xmax, ymax = gdf.total_bounds
    grid_x, grid_y = np.meshgrid(
        np.arange(xmin, xmax, resolution),
        np.arange(ymin, ymax, resolution)
    )

    # Step 3: Perform Natural Neighbor Interpolation
    raster = griddata(
        points=(x, y),  # Input points
        values=z,       # Depth values at input points
        xi=(grid_x, grid_y),  # Grid coordinates
        method="nearest"      # Natural Neighbor interpolation
    )

    # Step 4: Create raster transform
    transform = from_origin(xmin, ymax, resolution, resolution)

    # Step 5: Create a mask from polygon geometries
    mask = rasterize(
        [(geom, 1) for geom in gdf.geometry],
        out_shape=raster.shape,
        transform=transform,
        fill=0,
        dtype="uint8",
    )
    # Apply mask: Set values outside the polygons to NaN
    raster = np.where(mask == 1, raster, np.nan)

    # Step 6: Reproject the raster in memory to EPSG:4326
    src_crs = gdf.crs
    dst_crs = "EPSG:4326"

    # Define destination raster dimensions (same resolution, different CRS)
    with rasterio.Env():
        dst_transform, width, height = rasterio.warp.calculate_default_transform(
            src_crs, dst_crs, raster.shape[1], raster.shape[0], *gdf.total_bounds
        )
        dst_raster = np.empty((height, width), dtype="float32")

        reproject(
            source=raster,
            destination=dst_raster,
            src_transform=transform,
            src_crs=src_crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear,
        )

    # Step 7: Save the reprojected raster to GeoTIFF
    with rasterio.open(
        output_file,
        "w",
        driver="GTiff",
        height=dst_raster.shape[0],
        width=dst_raster.shape[1],
        count=1,
        dtype="float32",
        crs=dst_crs,
        transform=dst_transform,
        nodata=np.nan,
    ) as dst:
        dst.write(dst_raster, 1)

    print(f"Reprojected raster saved to {output_file}")


In [ ]:
test_natural_neighbor_with_mask(gdbinfo[surveynames[500]], f'/home/clay/Documents/SDB/test_{surveynames[500]}.tif')

In [ ]:
with rasterio.open(f'/home/clay/Documents/SDB/test_{surveynames[-2]}.tif') as src:
    raster = src.read(1)

plt.imshow(raster, cmap='viridis', extent=(xmin, xmax, ymin, ymax))
plt.colorbar(label="Depth (ft)")
plt.title("Rasterized Depth")
plt.show()

# Get the .xyz file from each download and apply kriging

In [ ]:
xyzinfo={}
for name in surveynames:
    folder_path = os.path.join(DOWNLOAD_DIR, name)
    xyzfile = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.XYZ')][0]
    gdb_file = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.gdb')][0]
    layers = fiona.listlayers(gdb_file)
    bathyvector = gpd.read_file(gdb_file, layer='Bathymetry_Vector')

    xyzinfo[name] = [xyzfile, bathyvector]

In [ ]:
# Step 1: Load the XYZ data
survey = surveynames[950]
xyzdata = pd.read_csv(xyzinfo[survey][0], sep='\\s+', header=None, names=["x", "y", "z"])

# Load the AOI GeoDataFrame (multipolygon shapefiles)
aoi_gdf = xyzinfo[survey][1]
aoi_gdf = aoi_gdf.to_crs(xyzinfo[survey][1].crs)  # Ensure AOI and .xyz share the same CRS

# Step 2: Clip the .xyz data to the AOI polygons
geometry = [Point(x, y) for x, y in zip(xyzdata["x"], xyzdata["y"])]
xyz_gdf = gpd.GeoDataFrame(xyzdata, geometry=geometry, crs=xyzinfo[survey][1].crs)
clipped_xyz_gdf = gpd.clip(xyz_gdf, aoi_gdf)  # Clip .xyz points to AOI polygons

# Extract the clipped points
clipped_xyz = clipped_xyz_gdf[["x", "y", "z"]]

# Step 3: Define the grid for interpolation
resolution = 10 * 3.28084  # Convert 10 meters to feet
x_min, x_max = clipped_xyz["x"].min(), clipped_xyz["x"].max()
y_min, y_max = clipped_xyz["y"].min(), clipped_xyz["y"].max()

x_grid = np.arange(x_min, x_max + resolution, resolution)
y_grid = np.arange(y_min, y_max + resolution, resolution)
grid_x, grid_y = np.meshgrid(x_grid, y_grid)

# Step 4: Perform Interpolation
raster = griddata(
    points=clipped_xyz[["x", "y"]].values,  # Correct slicing for DataFrame columns
    values=clipped_xyz["z"].values,
    xi=(grid_x, grid_y),
    method="linear"  # Interpolate between points
)

# Step 5: Mask the Raster with AOI Polygons
# Rasterize the AOI polygons
transform = from_origin(x_min, y_max, resolution, resolution)
aoi_mask = rasterize(
    [(geom, 1) for geom in aoi_gdf.geometry],
    out_shape=raster.shape,
    transform=transform,
    fill=255,  # Outside AOI is 0
    dtype="uint8",
)

# Apply the mask: Set values outside AOI to NaN
raster = np.where(aoi_mask == 1, raster, np.nan)

# Step 6: Visualize the Raster
plt.imshow(
    raster,
    extent=(x_min, x_max, y_min, y_max),
    origin="lower",
    cmap="viridis"
)
plt.colorbar(label="Depth (ft)")
plt.title("Rasterized Bathymetry (Clipped to AOI)")
plt.xlabel("X (ft)")
plt.ylabel("Y (ft)")
plt.show()

# Step 7: Save and Reproject the Raster to EPSG:4326
output_crs = "EPSG:4326"

# First, calculate the transform for the target CRS
dst_transform, width, height = calculate_default_transform(
    xyzinfo[survey][1].crs,  # Original CRS of the .xyz data
    output_crs,
    raster.shape[1],  # Width in pixels
    raster.shape[0],  # Height in pixels
    *[x_min, y_min, x_max, y_max],  # Bounds of the original raster
)

# Create an empty array for the reprojected raster
reprojected_raster = np.empty((height, width), dtype="float32")

# Perform the reprojection
with rasterio.Env():
    reproject(
        source=raster,
        destination=reprojected_raster,
        src_transform=transform,
        src_crs=xyzinfo[survey][1].crs,  # Original CRS
        dst_transform=dst_transform,
        dst_crs=output_crs,  # Target CRS
        resampling=Resampling.bilinear,  # Choose resampling method
    )

# Step 8: Rasterize the AOI in EPSG:4326
aoi_gdf_epsg4326 = aoi_gdf.to_crs(output_crs)  # Reproject AOI to EPSG:4326
aoi_mask_epsg4326 = rasterize(
    [(geom, 1) for geom in aoi_gdf_epsg4326.geometry],
    out_shape=(height, width),
    transform=dst_transform,
    fill=0,  # Outside AOI is 0
    dtype="uint8",
)

# Apply the mask to the reprojected raster
reprojected_raster = np.where(aoi_mask_epsg4326 == 1, reprojected_raster, np.nan)

# Save the reprojected raster to GeoTIFF
with rasterio.open(
    f'/home/clay/Documents/SDB/test_{survey}_epsg4326_clipped.tif',
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="float32",
    crs=output_crs,
    transform=dst_transform,
    nodata=np.nan,
) as dst:
    dst.write(reprojected_raster, 1)

In [ ]:

# Load the XYZ data
survey = surveynames[-20]
xyzdata = pd.read_csv(xyzinfo[survey][0], sep='\\s+', header=None, names=["x", "y", "z"])

# Define the CRS (original)
input_crs = xyzinfo[survey][1].crs

# Step 1: Create a Convex Hull of the Points (Boundary)
points = MultiPoint(xyzdata[["x", "y"]].values)
boundary = points.convex_hull  # Convex hull as a Shapely Polygon

# Convert the boundary to GeoJSON (optional visualization)
boundary_gdf = gpd.GeoDataFrame({"geometry": [boundary]}, crs=input_crs)

# Step 2: Define the Grid
resolution = 10 * 3.28084  # Convert 10 meters to feet
x_min, x_max = xyzdata["x"].min(), xyzdata["x"].max()
y_min, y_max = xyzdata["y"].min(), xyzdata["y"].max()

x_grid = np.arange(x_min, x_max + resolution, resolution)
y_grid = np.arange(y_min, y_max + resolution, resolution)
grid_x, grid_y = np.meshgrid(x_grid, y_grid)

# Step 3: Interpolate the Data
raster = griddata(
    points=xyzdata[["x", "y"]].values,
    values=xyzdata["z"].values,
    xi=(grid_x, grid_y),
    method="nearest"  # Interpolate between points
)

# Step 4: Rasterize the Boundary as a Mask
transform = from_origin(x_min, y_max, resolution, resolution)
boundary_mask = rasterize(
    [(boundary, 1)],  # Use the convex hull 
    out_shape=raster.shape,
    transform=transform,
    fill=0,  # Outside boundary is 0
    dtype="uint8",
)

# Step 5: Apply the Mask to the Raster
raster = np.where(boundary_mask == 1, raster, np.nan)  # Mask values outside the boundary

# Step 6: Visualize the Raster
plt.imshow(
    raster,
    extent=(x_min, x_max, y_min, y_max),
    origin="lower",
    cmap="viridis"
)
plt.colorbar(label="Depth (ft)")
plt.title("Rasterized Bathymetry (Clipped to Boundary)")
plt.xlabel("X (ft)")
plt.ylabel("Y (ft)")
plt.show()

In [ ]:
with rasterio.open(
    "output_raster_with_boundary.tif",
    "w",
    driver="GTiff",
    height=raster.shape[0],
    width=raster.shape[1],
    count=1,
    dtype="float32",
    crs=input_crs,
    transform=transform,
    nodata=np.nan,
) as dst:
    dst.write(raster, 1)

# Now, go to 01b_get_s2.ipynb to use the extent of the valid data in the created bathymetry rasters to get cloud-masked Sentinel-2 L2A products from Google Earth Engine.